## Trying to work out why there is a dvide by zero error in calculation of cross-contour transport


GPC015, year 2167 months 1 and 2

GPC018, year 2162 month 2 and year 2164 month 11?

In [1]:
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
import netCDF4 as nc
import xarray as xr
import numpy as np
from dask.distributed import Client
import cmocean as cm



In [2]:
client = Client(n_workers=4)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/37909/status,
Dashboard: /proxy/37909/status,Workers: 4
Total threads: 16,Total memory: 64.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33015,Workers: 4
Dashboard: /proxy/37909/status,Total threads: 16
Started: Just now,Total memory: 64.00 GiB
Comm: tcp://127.0.0.1:33849,Total threads: 4
Dashboard: /proxy/35065/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:34635,


In [8]:
master_session = cc.database.create_session()

control = '01deg_jra55v13_ryf9091'

#session_name = '/g/data/e14/pc5520/model_data/access-om2/basal_melt_MOM5/basal_melt_MOM5.db'
#session_name = '/g/data/v45/akm157/model_data/access-om2/basal_melt_MOM5/basal_melt_MOM5.db'
session_name ='/g/data/x77/cy8964/databases/basal_melt_MOM5_v2.db'
basal_melt_session = cc.database.create_session(session_name)

# temperature of meltwater using Gade line:
basal = 'accessom2-GPC015'
iceberg = 'accessom2-GPC018'


In [13]:
month = '02'
year = '2152'

first_year = year
last_year = year

start_time=first_year+'-'+month
end_time=last_year+'-'+month

# reference density value:
rho_0 = 1035.0
lat_range = slice(-90,-59)

In [14]:
expt = iceberg
session = basal_melt_session

In [15]:
###############################
# ### Open grid cell width data for domain

## some grid data is required, a little complicated because these variables don't behave well with some 
dyt = cc.querying.getvar(expt, 'dyt',session, n=1, ncfile = 'ocean_grid.nc')
dxu = cc.querying.getvar(expt, 'dxu',session, n=1, ncfile = 'ocean_grid.nc')

# select latitude range:
dxu = dxu.sel(yu_ocean=lat_range)
dyt = dyt.sel(yt_ocean=lat_range)

###############################
# ### Open contour data

isobath_depth = 1000
outfile = '/g/data/v45/akm157/model_data/access-om2/Antarctic_slope_contour_'+str(isobath_depth)+'m.npz'
data = np.load(outfile)
mask_y_transport = data['mask_y_transport']
mask_x_transport = data['mask_x_transport']
mask_y_transport_numbered = data['mask_y_transport_numbered']
mask_x_transport_numbered = data['mask_x_transport_numbered']

yt_ocean = cc.querying.getvar(expt,'yt_ocean',session,n=1)
yt_ocean = yt_ocean.sel(yt_ocean=lat_range)
yu_ocean = cc.querying.getvar(expt,'yu_ocean',session,n=1)
yu_ocean = yu_ocean.sel(yu_ocean=lat_range)
xt_ocean = cc.querying.getvar(expt,'xt_ocean',session,n=1)
xu_ocean = cc.querying.getvar(expt,'xu_ocean',session,n=1)

# Convert contour masks to data arrays, so we can multiply them later.
# We need to ensure the lat lon coordinates correspond to the actual data location:
#       The y masks are used for vhrho, so like vhrho this should have dimensions (yu_ocean, xt_ocean).
#       The x masks are used for uhrho, so like uhrho this should have dimensions (yt_ocean, xu_ocean).
#       However the actual name will always be simply y_ocean/x_ocean irrespective of the variable
#       to make concatenation of transports in both direction and sorting possible.

mask_x_transport = xr.DataArray(mask_x_transport, coords = [yt_ocean, xu_ocean], dims=['y_ocean', 'x_ocean'])
mask_y_transport = xr.DataArray(mask_y_transport, coords = [yu_ocean, xt_ocean], dims=['y_ocean', 'x_ocean'])
mask_x_transport_numbered = xr.DataArray(mask_x_transport_numbered, coords = [yt_ocean, xu_ocean], dims=['y_ocean', 'x_ocean'])
mask_y_transport_numbered = xr.DataArray(mask_y_transport_numbered, coords = [yu_ocean, xt_ocean], dims=['y_ocean', 'x_ocean'])

# ### Stack contour data into 1D

# Create the contour order data-array. Note that in this procedure the x-grid counts have x-grid
#   dimensions and the y-grid counts have y-grid dimensions, but these are implicit, the dimension 
#   *names* are kept general across the counts, the generic y_ocean, x_ocean, so that concatening works
#   but we dont double up with numerous counts for one lat/lon point.

# stack contour data into 1d:
mask_x_numbered_1d = mask_x_transport_numbered.stack(contour_index = ['y_ocean', 'x_ocean'])
mask_x_numbered_1d = mask_x_numbered_1d.where(mask_x_numbered_1d > 0, drop = True)
mask_y_numbered_1d = mask_y_transport_numbered.stack(contour_index = ['y_ocean', 'x_ocean'])
mask_y_numbered_1d = mask_y_numbered_1d.where(mask_y_numbered_1d > 0, drop = True)
contour_ordering = xr.concat((mask_x_numbered_1d,mask_y_numbered_1d), dim = 'contour_index')
contour_ordering = contour_ordering.sortby(contour_ordering)
contour_index_array = np.arange(1,len(contour_ordering)+1)

# get lat and lon along contour, useful for plotting later:
lat_along_contour = contour_ordering.y_ocean
lon_along_contour = contour_ordering.x_ocean
# don't need the multi-index anymore, replace with contour count
lat_along_contour.coords['contour_index'] = contour_index_array
lon_along_contour.coords['contour_index'] = contour_index_array


In [17]:
# ### Open uhrho, vhrho from daily data

# Note vhrho_nt is v*dz*1035 and is positioned on north centre edge of t-cell.
vhrho = cc.querying.getvar(expt,'vhrho_nt',session,ncfile='%daily%', start_time = '2152-01-01', end_time = '2153-03-31')
uhrho = cc.querying.getvar(expt,'uhrho_et',session,ncfile='%daily%', start_time = '2152-01-01', end_time = '2153-03-31')

# the coords on regional variables might be wacky, in which case need to do:
vhrho = vhrho.rename({'yt_ocean_sub01':'yt_ocean', 'xt_ocean_sub01':'xt_ocean'})
uhrho = uhrho.rename({'yt_ocean_sub01':'yt_ocean', 'xt_ocean_sub01':'xt_ocean'})
# if meridional length is too long, cut to match:
vhrho = vhrho[:,:,:len(yt_ocean),:]
uhrho = uhrho[:,:,:len(yt_ocean),:]
vhrho.coords['yt_ocean'] = yt_ocean
uhrho.coords['yt_ocean'] = yt_ocean

# select latitude range and this month:
vhrho = vhrho.sel(yt_ocean=lat_range).sel(time=slice(start_time,end_time))
uhrho = uhrho.sel(yt_ocean=lat_range).sel(time=slice(start_time,end_time))


Exception during reset or similar
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 763, in _finalize_fairy
    fairy._reset(pool, transaction_was_reset)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 1038, in _reset
    pool._dialect.do_rollback(self)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 683, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 22409659656000 and this is thread id 22408529401600.
Exception closing connection <sqlite3.Connection object at 0x146197a32d50>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.01/lib/python3.9/site-pa

In [18]:
vhrho

ValueError: Array does not contain datetime objects.

<xarray.DataArray 'vhrho_nt' (time: 0, st_ocean: 75, yt_ocean: 510,
                              xt_ocean: 3600)>
dask.array<getitem, shape=(0, 75, 510, 3600), dtype=float32, chunksize=(0, 75, 36, 45), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... -59.14 -59.08 -59.03
  * st_ocean  (st_ocean) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * time      (time) object 
Attributes:
    long_name:      vhrho_nt on horz face of T-cells
    units:          (kg/m^3)*m^2/sec
    valid_range:    [-1000000.  1000000.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    ncfiles:        ['/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/a...

2023-03-16 15:13:31,267 - distributed.nanny - WARNING - Restarting worker
2023-03-16 15:13:31,359 - distributed.nanny - WARNING - Restarting worker
2023-03-16 15:13:31,663 - distributed.nanny - WARNING - Restarting worker
2023-03-16 15:13:32,669 - distributed.nanny - WARNING - Restarting worker


## File corrupted??

Above code words fine for `month = '03'`